In [492]:
import requests 
import re as regex
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import aries_cloudagent.wallet.util as util
import base64
import base58
import uuid

agent1 = {}
agent2 = {}
plugin = {}

## Enter Invite Url:

In [493]:
agent1['url'] = "http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMjJmMmY0MGQtMDNiNC00Yzc0LWJlOTEtNjVmZDU3NzIwMDRkIiwgInJlY2lwaWVudEtleXMiOiBbIkVjNHV1cE5ZcGZ5eHFOU2Y5TFh5a05vZ0ptTTZ0aHFVTHY5cUh2QjFxTnVxIl0sICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovL2FnZW50Mi5sb2NhbGhvc3QiLCAibGFiZWwiOiAiQ2xpZW50IChhZG1pbikifQ=="

In [494]:
# Process invite url, delete white spaces
agent1['url'] = agent1['url'].replace(" ", "")
# Regex(substitution) to extract only the invite string from url
agent1['invite_string_b64'] = regex.sub(
           r".*(c\_i\=)", 
           r"", 
           agent1['url'])
# Decoding invite string using base64 decoder
agent1['invite_string'] = base64.b64decode(agent1['invite_string_b64'])
# Converting our invite json string into a dictionary 
agent1['invite'] = json.loads(agent1['invite_string'])

print("Decoded invitation:\n")
pp.pprint(agent1['invite'])

Decoded invitation:

{'@id': '22f2f40d-03b4-4c74-be91-65fd5772004d',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['Ec4uupNYpfyxqNSf9LXykNogJmM6thqULv9qHvB1qNuq'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [495]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

# print(base58.b58encode(did))
# print(plugin['did'])
print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 2BN9notxSeAyRkBCgkyuB8gQzhKik6xCuc4YvC9hTc2cu1AkndX19zLME7iwYtMzhXSE453JHKKCXFGaa6XRokCM

Public key: 
 9RokKfrREJvJEV8Z7f9CombqpjK6yhmJHbPm8AsiazYD


**Sending a connection request to acapy**

In [496]:
uniqueId = str(uuid.uuid4())
# print(uniqueId)

# our request body
message = {
        "@id":  uniqueId,
        "~transport": {
          "return_route": "all"
        },
        "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/request",
        "label": "Plugin",
        "connection": {
          "DID": plugin['did'],
          "DIDDoc": {
            "@context": "https://w3id.org/did/v1",
            "id": plugin['did'],
            "publicKey": [{
              "id": plugin['did'] + "#keys-1",
              "type": "Ed25519VerificationKey2018",
              "controller": plugin['did'],
              "publicKeyBase58": plugin['public_key_b58']
            }],
            "service": [{
              "id": plugin['did'] + ";indy",
              "type": "IndyAgent",
              "recipientKeys": plugin['public_key_b58'],
              "serviceEndpoint": ""
            }]
          }
        }
      }

**Encoding/packing the request**

In [497]:
# encoding it with aries crypto function using the key that was
# given to us by aca-py in recipientKeys
decodedAcapyKey = base58.b58decode(agent1['invite']['recipientKeys'][0])
ourPrivateKey = plugin['keypair'][1]
# print(plugin['did'])
encodedMessage = \
    crypto.encode_pack_message(json.dumps(message), [decodedAcapyKey], ourPrivateKey)

encodedMessage = encodedMessage.decode("ascii")
# print("Encoded message: \n")
# pp.pprint(encodedMessage)

In [498]:
connectionRequestResponse = requests.post(agent1['invite']['serviceEndpoint'], data=encodedMessage)
assert connectionRequestResponse.text != "", "invalid response from acapy"

**Response from aca-py**

In [499]:
# a bit of a hack to simplify message unpacking,
# decode_pack_message needs a callable object for some reason
def unpackMessage(message, privateKey):
    class FindKey:
        def __init__(self, key):
            self.key = key

        def __call__(self, argument):
            return self.key

    find_key = FindKey(privateKey)   
    return crypto.decode_pack_message(message, find_key)
    
connectionRequestResponseUnpacked = \
    unpackMessage(connectionRequestResponse.text, plugin['keypair'][1])

connectionRequestResponseDict = json.loads(connectionRequestResponseUnpacked[0])

print("Decoded acapy response: \n")
pp.pprint(connectionRequestResponseUnpacked)

Decoded acapy response: 

('{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/response", '
 '"@id": "33c62541-bf41-4b30-b0ef-277d54758831", "~thread": {"thid": '
 '"ed569bb8-bbdf-41c1-aa59-9b0c6ab83c82"}, "connection~sig": {"@type": '
 '"did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/signature/1.0/ed25519Sha512_single", '
 '"signature": '
 '"3Rgy6TiH6f33n9jWyrth7TEzPlk5PNMIjWp1BK3BjCjZSirh80XUqzZsWhXyCxVFBRd0yIOlfrsb01koz-p9DA==", '
 '"sig_data": '
 '"AAAAAF69IdZ7IkRJRCI6ICJLWW81Q0Rlc1NKQ2dQVXZKUHV3ZjZrIiwgIkRJRERvYyI6IHsiQGNvbnRleHQiOiAiaHR0cHM6Ly93M2lkLm9yZy9kaWQvdjEiLCAiaWQiOiAiZGlkOnNvdjpLWW81Q0Rlc1NKQ2dQVXZKUHV3ZjZrIiwgInB1YmxpY0tleSI6IFt7ImlkIjogImRpZDpzb3Y6S1lvNUNEZXNTSkNnUFV2SlB1d2Y2ayMxIiwgInR5cGUiOiAiRWQyNTUxOVZlcmlmaWNhdGlvbktleTIwMTgiLCAiY29udHJvbGxlciI6ICJkaWQ6c292OktZbzVDRGVzU0pDZ1BVdkpQdXdmNmsiLCAicHVibGljS2V5QmFzZTU4IjogIkI3THlyUm1xdXN2TXpidU5naFllWnlkQ0NBNmUzdVBETnZqV2FQazgzd1dSIn1dLCAiYXV0aGVudGljYXRpb24iOiBbeyJ0eXBlIjogIkVkMjU1MTlTaWduYXR1cmVBdXRoZW50aWNhdGlvbjIwMT

**Unpacking connection data embedded in the response**

In [500]:
sig_data_raw = connectionRequestResponseDict['connection~sig']['sig_data']
sig_data_raw = base64.b64decode(sig_data_raw)
## TODO: replace - _ characters with some random number etc.
# avoid first 8 characters as they are a time signature
sig_data_raw = sig_data_raw[8:]
sig_data_raw = sig_data_raw.decode('ascii')

sig_data = json.loads(sig_data_raw)
pp.pprint(sig_data)

{'DID': 'KYo5CDesSJCgPUvJPuwf6k',
 'DIDDoc': {'@context': 'https://w3id.org/did/v1',
            'authentication': [{'publicKey': 'did:sov:KYo5CDesSJCgPUvJPuwf6k#1',
                                'type': 'Ed25519SignatureAuthentication2018'}],
            'id': 'did:sov:KYo5CDesSJCgPUvJPuwf6k',
            'publicKey': [{'controller': 'did:sov:KYo5CDesSJCgPUvJPuwf6k',
                           'id': 'did:sov:KYo5CDesSJCgPUvJPuwf6k#1',
                           'publicKeyBase58': 'B7LyrRmqusvMzbuNghYeZydCCA6e3uPDNvjWaPk83wWR',
                           'type': 'Ed25519VerificationKey2018'}],
            'service': [{'id': 'did:sov:KYo5CDesSJCgPUvJPuwf6k;indy',
                         'priority': 0,
                         'recipientKeys': ['B7LyrRmqusvMzbuNghYeZydCCA6e3uPDNvjWaPk83wWR'],
                         'serviceEndpoint': 'http://agent2.localhost',
                         'type': 'IndyAgent'}]}}


**Adding the connection to connection list**

In [501]:
connections = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": sig_data['DIDDoc'],
        "myKey": plugin['keypair'][1]
    }
]

def packMessage(message, connection):
    # pass in our private key and recipient key to the encode_pack_message
    decodedRecipientKey = base58.b58decode(connection['DIDDoc']['service'][0]['recipientKeys'][0])
    packedMessage = \
    crypto.encode_pack_message(json.dumps(message), \
                               [decodedRecipientKey], \
                               connection['myKey'])
    return packedMessage.decode('ascii')


pp.pprint(connections)

[{'DIDDoc': {'@context': 'https://w3id.org/did/v1',
             'authentication': [{'publicKey': 'did:sov:KYo5CDesSJCgPUvJPuwf6k#1',
                                 'type': 'Ed25519SignatureAuthentication2018'}],
             'id': 'did:sov:KYo5CDesSJCgPUvJPuwf6k',
             'publicKey': [{'controller': 'did:sov:KYo5CDesSJCgPUvJPuwf6k',
                            'id': 'did:sov:KYo5CDesSJCgPUvJPuwf6k#1',
                            'publicKeyBase58': 'B7LyrRmqusvMzbuNghYeZydCCA6e3uPDNvjWaPk83wWR',
                            'type': 'Ed25519VerificationKey2018'}],
             'service': [{'id': 'did:sov:KYo5CDesSJCgPUvJPuwf6k;indy',
                          'priority': 0,
                          'recipientKeys': ['B7LyrRmqusvMzbuNghYeZydCCA6e3uPDNvjWaPk83wWR'],
                          'serviceEndpoint': 'http://agent2.localhost',
                          'type': 'IndyAgent'}]},
  'label': 'Client (admin)',
  'myKey': b':\xf4\x81\xdd\xa2\xf16\x8c\xa7\xd6JB\xdb\xd5\x85\xf8'


### FEATURE DISCOVERY

In [502]:
message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/query"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
message['query'] = '*'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'caff7deb-9a02-4e05-b8dd-ea11c8c14812',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/disclose',
 'protocols': [{'pid': 'https://didcomm.org/routing/1.0'},
               {'pid': 'https://didcomm.org/connections/1.0'},
               {'pid': 'https://didcomm.org/discover-features/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/notification/1.0'},
               {'pid': 'https://didcomm.org/introduction-service/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/credential-presentation/0.1'},
               {'pid': 'https://didcomm.org/notification/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-static-connections/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/introduction-service/0.1'},
               {'pid': 'https://didcomm.org/credential-issuance/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-issuer/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrj

## Dids

In [503]:
message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-list-dids"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'd9342669-98fa-4b06-88fb-ed5dd75aa78a',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/list-dids',
 'result': [{'did': 'SRypyC6EGLfDQrXMkLB78s',
             'verkey': 'Erv25DwntLu9Wrv9qDDpTyT5geAP2EFJKhoCEaa59wy7'},
            {'did': 'R7jMcbdYSWDBZ6iWqsTbXP',
             'verkey': 'E9Md2W85gvBbLM471Bok4PAm8skfZwHQJxEW33u2P4GL'},
            {'did': 'GfMEYgBHWeap9Y1TNBkWs3',
             'verkey': '9Y5hV3wHxTZv9HMaJHHKAmBEzmcKLX3R2Go4TNnNE3b4'},
            {'did': 'LcHB9CYg1ubfTfZfqgioMc',
             'verkey': 'BgrdPGjy34iF6HpX3ADondV8Dc92T7BkHv7CCuYwwVSY'},
            {'did': 'GA5cdEpPbyxcWyLmUGZqNX',
             'verkey': '9G8SPcRmoaepYaLKY1UhzZF1EjXe7vmKcHhVMmJmRgj5'},
            {'did': 'UuzotrWcFJ7Prz3LmRGJbB',
             'verkey': 'GDQgHUQvL827rwJt9GNCzRyfUFUASQuhr8294NDjHfiR'},
            {'did': 'A16AZUPifEcvmYjhUdyJQm',
             'verkey': '5uZGKL4EhHvre87MJKwyRHUoYt5ZZuRg1YrXMNkCUGMZ'},
            {'did': '35G1tpBSY1uXYuDUU3eFmp',
      

## Public did

In [504]:
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-public-did"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '6fe2acda-50ff-4dbf-9242-8c4966d16e2c',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 '~thread': {'thid': 'a368120d-5a12-46df-8ced-58c7b1e21d0e'}}


In [509]:
message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/query"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
message['comment'] = 'qqq'

# message['comment'] = "memes"
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'fef40c76-40b8-4b13-b607-e72283e89e66',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/query',
 'comment': 'qqq',
 '~thread': {'thid': '8be8effd-7b88-4728-937d-1b704789f700'}}
